In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    import networkx as nx
except ImportError:
    print("Note: This notebook uses the 'networkx' library. Install it with: pip install networkx")


In [ ]:
# 1. Erdős–Rényi Random Graph (G(n, p))

n = 200          # number of nodes
p = 0.05         # connection probability

G_er = nx.erdos_renyi_graph(n, p, seed=42)

print(f"Number of nodes: {G_er.number_of_nodes()}")
print(f"Number of edges: {G_er.number_of_edges()}")

plt.figure()
nx.draw_spring(G_er, node_size=20, linewidths=0.2, width=0.2)
plt.title("Erdős–Rényi Random Graph (n=200, p=0.05)")
plt.axis("off")
plt.show()


In [ ]:
# 2. Degree Distribution of the Random Graph

degrees = [deg for _, deg in G_er.degree()]
unique_deg, counts = np.unique(degrees, return_counts=True)

plt.figure()
plt.bar(unique_deg, counts / counts.sum())
plt.xlabel("Degree k")
plt.ylabel("P(k)")
plt.title("Degree Distribution of Erdős–Rényi Graph")
plt.show()


In [ ]:
# 3. Watts–Strogatz Small-World Network

n = 200     # nodes
k = 6       # each node connected to k nearest neighbours
beta = 0.1  # rewiring probability

G_sw = nx.watts_strogatz_graph(n, k, beta, seed=42)

plt.figure()
nx.draw_spring(G_sw, node_size=20, linewidths=0.2, width=0.2)
plt.title("Watts–Strogatz Small-World Network (n=200, k=6, β=0.1)")
plt.axis("off")
plt.show()


In [ ]:
# 4. Compare Average Path Length and Clustering: Random vs Small-World

def safe_average_shortest_path_length(G):
    if nx.is_connected(G):
        return nx.average_shortest_path_length(G)
    else:
        # take the largest connected component
        largest_cc = max(nx.connected_components(G), key=len)
        subG = G.subgraph(largest_cc)
        return nx.average_shortest_path_length(subG)

L_er = safe_average_shortest_path_length(G_er)
C_er = nx.average_clustering(G_er)

L_sw = safe_average_shortest_path_length(G_sw)
C_sw = nx.average_clustering(G_sw)

print("Erdős–Rényi random graph:")
print(f"  Average shortest path length ≈ {L_er:.3f}")
print(f"  Clustering coefficient ≈ {C_er:.3f}")

print("\nWatts–Strogatz small-world network:")
print(f"  Average shortest path length ≈ {L_sw:.3f}")
print(f"  Clustering coefficient ≈ {C_sw:.3f}")


In [ ]:
# 5. Simple SI Spreading Process on a Network

def si_spread(G, beta=0.05, t_max=50, initial_fraction=0.01, seed=0):
    rng = np.random.default_rng(seed)
    N = G.number_of_nodes()
    nodes = list(G.nodes())

    # 0 = susceptible, 1 = infected
    state = np.zeros(N, dtype=int)

    # randomly infect a small fraction
    initial_infected = rng.choice(N, size=max(1, int(initial_fraction * N)), replace=False)
    state[initial_infected] = 1

    infected_counts = [state.sum()]

    for t in range(t_max):
        new_state = state.copy()
        for i in range(N):
            if state[i] == 1:
                # infect susceptible neighbours with prob beta
                for j in G.neighbors(nodes[i]):
                    idx_j = nodes.index(j)
                    if state[idx_j] == 0 and rng.random() < beta:
                        new_state[idx_j] = 1
        state = new_state
        infected_counts.append(state.sum())

    return np.arange(t_max + 1), np.array(infected_counts) / N

t, frac_inf_er = si_spread(G_er, beta=0.03, t_max=60, initial_fraction=0.02, seed=1)
t, frac_inf_sw = si_spread(G_sw, beta=0.03, t_max=60, initial_fraction=0.02, seed=1)

plt.figure()
plt.plot(t, frac_inf_er, label="Erdős–Rényi")
plt.plot(t, frac_inf_sw, label="Small-world", linestyle="--")
plt.xlabel("Time step")
plt.ylabel("Fraction infected")
plt.title("SI Spreading Process on Networks")
plt.legend()
plt.show()
